In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers  as KL
import tensorflow.keras.models  as KM

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
#normalizing train and test data
x_train, x_test = np.expand_dims(x_train, axis=-1), np.expand_dims(x_test, axis=-1)
#reshaping the train and test data as cnn accepts only 4D array

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
train_encoded = y_train.reshape(-1,1)
Y_train = ohe.fit_transform(train_encoded)

test_encoded = y_test.reshape(-1,1)
Y_test = ohe.fit_transform(test_encoded)

In [4]:
x_train.dtype

dtype('float64')

In [5]:
inputs = KL.Input(shape=(28, 28, 1))
c = KL.Conv2D(32, (3, 3), padding="valid", activation=tf.nn.relu)(inputs)
# input vol = 28x28x1
# filter = 32(3x3)
# o/p vol = (w1-f+2p)/s+1
# o/p vol = (28-3)/1+1=26
# 32(3*3*1+1) = 320
m = KL.MaxPool2D((2, 2), (2, 2))(c)

# i/p vol = 26x26x32
# o/p vol = (26-2)/2+1=13

d = KL.Dropout(0.25)(m)
#Dropout randomly turns off neurons so that the model finds different pathways everytime
c = KL.Conv2D(64, (3, 3), padding="valid", activation=tf.nn.relu)(d)
# input vol = 13x13x32
# filter = 64(3x3)
# o/p vol = (13-3)/2+1=11x11x64
#((m * n * d)+1)* k)
#((3x3x32)+1x64)= 18496  
m = KL.MaxPool2D((2, 2), (2, 2))(c)
# input vol = 11x11x64
# o/p vol = (11-3)/2+1=5*5*64
c = KL.Conv2D(128, (3, 3), padding="valid", activation=tf.nn.relu)(m)
# input vol = 5x5x64
# filter = 128(3x3)
# o/p vol = (5-3)/1+1=3*3*128
#((m * n * d)+1)* k)
#((3x3x64)+1x128)=  73856   
m = KL.MaxPool2D((2, 2), (2, 2))(c)
f = KL.Flatten()(m)
#it flattens the the feature map
outputs = KL.Dense(10, activation=tf.nn.softmax)(f)
#(10*no.of neurons in the prev layer)+1(bias)*10)= 1290

model = KM.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856 

In [7]:
import tensorflow.keras.optimizers 
import tensorflow.keras.losses
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001,decay=1e-5)
losses = tensorflow.keras.losses.categorical_crossentropy

model.compile(optimizer = opt,
              loss=losses, metrics = ['accuracy'])
model.fit(x_train, Y_train, batch_size=64, epochs=3, validation_data=(x_test, Y_test))

test_loss, test_acc = model.evaluate(x_test, Y_test)
print("Test Loss: {0} - Test Acc: {1}".format(test_loss, test_acc))

Epoch 1/3
938/938 [==============================] - 55s 58ms/step - loss: 0.0561 - accuracy: 0.9824 - val_loss: 0.0542 - val_accuracy: 0.9829
Epoch 2/3
938/938 [==============================] - 55s 58ms/step - loss: 0.0433 - accuracy: 0.9862 - val_loss: 0.0480 - val_accuracy: 0.9846
Epoch 3/3
313/313 [==============================] - 3s 8ms/step - loss: 0.0480 - accuracy: 0.9853
Test Loss: 0.048029039055109024 - Test Acc: 0.9853000044822693
